In [87]:
import os
import pandas as pd
import numpy as np

IMPORTING PROCESSED RAW DATA

In [88]:
#DIRECTORIES OF DATASETS
# 1. Cabot_517
Cabot_517_combined_data = r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\517\combined_517.parquet'
Cabot_517_output_dir = r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\517'
Cabot_517_ID = '517'

# 2. Dolomite_499
Dolomite_499_combined_data = r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\499\combined_499.parquet'
Dolomite_499_output_dir = r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\499'
Dolomite_499_ID = '499'

# 3. Stella_494
Stella_494_combined_data = r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\494\combined_494.parquet'
Stella_494_output_dir = r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\494'
Stella_494_ID = '494'

# 4. Kelvin_481
Kelvin_481_combined_data = r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\481\combined_481.parquet'
Kelvin_481_output_dir = r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\481'
Kelvin_481_ID = '481'

# 5. Cabot_454
Cabot_454_combined_data = r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\454\combined_454.parquet'
Cabot_454_output_dir = r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\454'
Cabot_454_ID = '454'

In [110]:
combined_517_raw_data = pd.read_parquet(Cabot_517_combined_data)
combined_499_raw_data = pd.read_parquet(Dolomite_499_combined_data)
combined_494_raw_data = pd.read_parquet(Stella_494_combined_data)
combined_481_raw_data = pd.read_parquet(Kelvin_481_combined_data)
combined_454_raw_data = pd.read_parquet(Cabot_454_combined_data)

In [90]:
combined_494_raw_data

,c_ballast_pumped,c_battpos,c_climb_target_depth,c_de_oil_vol,c_dive_target_depth,c_heading,c_wpt_lat,c_wpt_lon,m_altitude,m_ballast_pumped,...,sci_oxy4_oxygen,sci_oxy4_timestamp,sci_satpar_par,sci_satpar_raw_counts,sci_satpar_timestamp,sci_water_cond,sci_water_pressure,sci_water_temp,c_thruster_on,m_thruster_power
0,None,None,None,None,None,None,None,None,0,None,...,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443563,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1443564,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1443565,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1443566,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


FILTERING DIVES WITH LESS THAN 10 POINTS IN EACH SENSOR

In [111]:
def filter_dataframe(df, output_dir, ID, columns_to_keep, cols_to_move):
    """
    Filters a combined DataFrame by keeping only the necessary columns, reordering columns,
    and filtering groups with at least 10 non-NA values in all columns.

    Args:
        df (pd.DataFrame): The combined DataFrame to filter.
        output_dir (str): Directory to save the filtered data.
        ID (str): Name of the output file (without extension).
        columns_to_keep (list): List of columns to keep in the DataFrame.
        cols_to_move (list): List of columns to reorder at the beginning of the DataFrame.

    Returns:
        pd.DataFrame: The filtered DataFrame.
    """
    # Keep only the specified columns
    df = df[columns_to_keep]

    # Reorder columns
    for col in reversed(cols_to_move):
        if col in df.columns:
            col_to_move = df.pop(col)
            df.insert(0, col, col_to_move)

    # Filter groups where all columns (except grouping ones) have at least 10 non-NA points
    filtered_df = df.groupby(['year', 'julian_day', 'dive_cycle']).filter(
        lambda group: all(group[col].notna().sum() >= 10 for col in group.columns)
    )

    # Define output file paths
    output_csv = f"{output_dir}/filtered_{ID}.csv"
    output_parquet = f"{output_dir}/filtered_{ID}.parquet"

    # Save the filtered data to CSV and Parquet
    filtered_df.to_csv(output_csv, index=False)
    filtered_df.to_parquet(output_parquet, index=False)

    return filtered_df



In [116]:
columns_to_keep = ['year', 'julian_day', 'dive_cycle', 'm_altitude', 'm_ballast_pumped', 'm_battery', 'm_battery_inst', 'm_battpos', 
                   'm_coulomb_amphr_total', 'm_depth', 'm_final_water_vx', 'm_final_water_vy', 'm_heading',
                   'm_lat', 'm_lon', 'm_leakdetect_voltage', 'm_leakdetect_voltage_forward', 'm_pitch', 'm_present_time', 
                   'm_roll', 'm_speed', 'sci_ctd41cp_timestamp', 'sci_flbbcd_bb_units', 'sci_flbbcd_cdom_units', 'sci_flbbcd_chlor_units',
                   'sci_m_present_time', 'sci_oxy4_oxygen', 'sci_water_cond', 'sci_water_pressure', 'sci_water_temp']

cols_to_move = ['year', 'julian_day', 'dive_cycle', 'm_present_time']

In [117]:
filtered_517 = filter_dataframe(combined_517_raw_data, Cabot_517_output_dir, Cabot_517_ID, columns_to_keep, cols_to_move)
filtered_499 = filter_dataframe(combined_499_raw_data, Dolomite_499_output_dir, Dolomite_499_ID, columns_to_keep, cols_to_move)
filtered_494 = filter_dataframe(combined_494_raw_data, Stella_494_output_dir, Stella_494_ID, columns_to_keep, cols_to_move)
filtered_481 = filter_dataframe(combined_481_raw_data, Kelvin_481_output_dir, Kelvin_481_ID, columns_to_keep, cols_to_move)
filtered_454 = filter_dataframe(combined_454_raw_data, Cabot_454_output_dir, Cabot_454_ID, columns_to_keep, cols_to_move)

filtered_517

,year,julian_day,dive_cycle,m_present_time,m_altitude,m_ballast_pumped,m_battery,m_battery_inst,m_battpos,m_coulomb_amphr_total,...,m_speed,sci_ctd41cp_timestamp,sci_flbbcd_bb_units,sci_flbbcd_cdom_units,sci_flbbcd_chlor_units,sci_m_present_time,sci_oxy4_oxygen,sci_water_cond,sci_water_pressure,sci_water_temp
211,2019,70,6,1552383986.10593,24.9499,None,None,None,None,None,...,0.555844,None,None,None,None,None,None,None,None,None
212,2019,70,6,1552384014.39502,None,None,None,14.7593,None,None,...,None,None,None,None,None,None,None,None,None,None
213,2019,70,6,1552384039.76459,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
214,2019,70,6,1552384044.15979,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
215,2019,70,6,1552384070.05399,None,None,None,None,None,None,...,0.389492,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936634,2019,118,0,None,None,None,None,None,None,None,...,None,None,None,None,None,1556838131.41394,None,None,None,None
1936635,2019,118,0,None,None,None,None,None,None,None,...,None,None,0.00154729,1.8018,1.1175,1556838140.53528,None,None,None,None
1936636,2019,118,0,None,None,None,None,None,None,None,...,None,None,None,None,None,1556838144.56229,342.983,None,None,None
1936637,2019,118,0,None,None,None,None,None,None,None,...,None,1556838135.01205,None,None,None,1556838145.57074,None,3.72464,0.079,9.3495


In [119]:
filtered_494

,year,julian_day,dive_cycle,m_present_time,m_altitude,m_ballast_pumped,m_battery,m_battery_inst,m_battpos,m_coulomb_amphr_total,...,m_speed,sci_ctd41cp_timestamp,sci_flbbcd_bb_units,sci_flbbcd_cdom_units,sci_flbbcd_chlor_units,sci_m_present_time,sci_oxy4_oxygen,sci_water_cond,sci_water_pressure,sci_water_temp
2810,2018,37,4,1518023838.48932,10.4762,None,None,None,None,None,...,0,None,None,None,None,None,None,None,None,None
2811,2018,37,4,1518023871.3446,None,None,None,10.9464,None,None,...,None,None,None,None,None,None,None,None,None,None
2812,2018,37,4,1518023890.70993,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2813,2018,37,4,1518023900.48114,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2814,2018,37,4,1518023925.28735,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439348,2018,123,3,1525490408.20203,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1439349,2018,123,3,1525490429.32858,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1439350,2018,123,3,1525490439.26947,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1439351,2018,123,3,None,None,None,None,None,None,None,...,None,0,0,0,0,1525490236.76218,0,0,0,0


TIME SYNCHRONIZATION

In [155]:
time_sync_517 = filtered_517.copy()
time_sync_499 = filtered_499.copy()
time_sync_494 = filtered_494.copy()
time_sync_481 = filtered_481.copy()
time_sync_454 = filtered_454.copy()
time_sync_454

,year,julian_day,dive_cycle,m_present_time,m_altitude,m_ballast_pumped,m_battery,m_battery_inst,m_battpos,m_coulomb_amphr_total,...,m_speed,sci_ctd41cp_timestamp,sci_flbbcd_bb_units,sci_flbbcd_cdom_units,sci_flbbcd_chlor_units,sci_m_present_time,sci_oxy4_oxygen,sci_water_cond,sci_water_pressure,sci_water_temp
502,2018,127,5,1525774207.36316,0,None,None,None,None,None,...,0,None,None,None,None,None,None,None,None,None
503,2018,127,5,1525774243.50058,None,None,None,11.1039,None,None,...,None,None,None,None,None,None,None,None,None,None
504,2018,127,5,1525774258.03674,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
505,2018,127,5,1525774273.14651,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
506,2018,127,5,1525774288.59177,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2119836,2018,190,0,None,None,None,None,None,None,None,...,None,None,None,None,None,1534162753.14557,None,4.56641,0.208,17.6559
2119837,2018,190,0,None,None,None,None,None,None,None,...,None,1534162754.53159,None,None,None,1534162760.17297,None,None,None,None
2119838,2018,190,0,None,None,None,None,None,None,None,...,None,None,0.0001512,-0.0863,0.3381,1534162762.18744,304.012,None,None,None
2119839,2018,190,0,None,None,None,None,None,None,None,...,None,None,None,None,None,1534162764.19357,None,4.56621,0.19,17.6565


In [156]:
# Function to convert time columns to numeric and synchronize time
def synchronize_time(df, time_columns):
    df[time_columns] = df[time_columns].apply(pd.to_numeric, errors='coerce')
    first_m_time = df.groupby(['year', 'julian_day', 'dive_cycle'])['m_present_time'].transform('first')
    df['time_present'] = df['m_present_time'] - first_m_time
    df['sci_time'] = df['sci_m_present_time'] - first_m_time
    df['sci_ctd_time'] = df['sci_ctd41cp_timestamp'] - first_m_time
    return df

# Synchronize time for all groups
time_sync_517 = synchronize_time(time_sync_517, ['m_present_time', 'sci_m_present_time', 'sci_ctd41cp_timestamp'])
time_sync_499 = synchronize_time(time_sync_499, ['m_present_time', 'sci_m_present_time', 'sci_ctd41cp_timestamp'])
time_sync_494 = synchronize_time(time_sync_494, ['m_present_time', 'sci_m_present_time', 'sci_ctd41cp_timestamp'])
time_sync_481 = synchronize_time(time_sync_481, ['m_present_time', 'sci_m_present_time', 'sci_ctd41cp_timestamp'])
time_sync_454 = synchronize_time(time_sync_454, ['m_present_time', 'sci_m_present_time', 'sci_ctd41cp_timestamp'])

time_sync_454

,year,julian_day,dive_cycle,m_present_time,m_altitude,m_ballast_pumped,m_battery,m_battery_inst,m_battpos,m_coulomb_amphr_total,...,sci_flbbcd_cdom_units,sci_flbbcd_chlor_units,sci_m_present_time,sci_oxy4_oxygen,sci_water_cond,sci_water_pressure,sci_water_temp,time_present,sci_time,sci_ctd_time
502,2018,127,5,1.525774e+09,0,None,None,None,None,None,...,None,None,NaN,None,None,None,None,0.000000e+00,NaN,NaN
503,2018,127,5,1.525774e+09,None,None,None,11.1039,None,None,...,None,None,NaN,None,None,None,None,3.613742e+01,NaN,NaN
504,2018,127,5,1.525774e+09,None,None,None,None,None,None,...,None,None,NaN,None,None,None,None,5.067358e+01,NaN,NaN
505,2018,127,5,1.525774e+09,None,None,None,None,None,None,...,None,None,NaN,None,None,None,None,6.578335e+01,NaN,NaN
506,2018,127,5,1.525774e+09,None,None,None,None,None,None,...,None,None,NaN,None,None,None,None,8.122861e+01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2119836,2018,190,0,NaN,None,None,None,None,None,None,...,None,None,1.534163e+09,None,4.56641,0.208,17.6559,NaN,2.943046e+06,NaN
2119837,2018,190,0,NaN,None,None,None,None,None,None,...,None,None,1.534163e+09,None,None,None,None,NaN,2.943053e+06,2.943047e+06
2119838,2018,190,0,NaN,None,None,None,None,None,None,...,-0.0863,0.3381,1.534163e+09,304.012,None,None,None,NaN,2.943055e+06,NaN
2119839,2018,190,0,NaN,None,None,None,None,None,None,...,None,None,1.534163e+09,None,4.56621,0.19,17.6565,NaN,2.943057e+06,NaN


Create new dataframe for each group, merging time to time_present if required.

In [157]:
# Function to load and process the dataset
def process_dataset(time_sync, m_present_group, sci_time_group, sci_CTD_time_group):
    # For m_present
    m_present_df = time_sync[m_present_group].copy()
    m_present_df = m_present_df.dropna(subset=['time_present'])

    # For sci_time_group
    sci_time_df = time_sync[sci_time_group].copy()
    sci_time_df['time_present'] = sci_time_df['time_present'].combine_first(sci_time_df.pop('sci_time')).clip(lower=0)
    sci_time_df.dropna(subset=['time_present'], inplace=True)

    # For sci_ctd_time_group
    ctd_time_df = time_sync[sci_CTD_time_group].copy()
    ctd_time_df['time_present'] = ctd_time_df['time_present'].combine_first(ctd_time_df.pop('sci_ctd_time')).clip(lower=0)
    ctd_time_df.dropna(subset=['time_present'], inplace=True)

    # Add rows with time intervals of 5s
    def add_time_rows(group):
        time_range = np.arange(0, group['time_present'].max() + 5, 5)  # Time steps of 5
        new_rows = pd.DataFrame({'time_present': time_range})
        for col in ['year', 'julian_day', 'dive_cycle']:
            new_rows[col] = group[col].iloc[0]
        combined = pd.concat([group, new_rows]).drop_duplicates(subset='time_present').sort_values('time_present')
        return combined.reset_index(drop=True)

    # Apply to each group
    m_present_df = m_present_df.groupby(['year', 'julian_day', 'dive_cycle'], group_keys=False).apply(add_time_rows)
    sci_time_df = sci_time_df.groupby(['year', 'julian_day', 'dive_cycle'], group_keys=False).apply(add_time_rows)
    ctd_time_df = ctd_time_df.groupby(['year', 'julian_day', 'dive_cycle'], group_keys=False).apply(add_time_rows)

    # Interpolation function
    def interpolate_data(df, columns_to_keep=['year', 'julian_day', 'dive_cycle', 'time_present']):
        df = df.dropna(subset=['time_present'])
        df = df.set_index('time_present')
        columns_to_interpolate = [col for col in df.columns if col not in columns_to_keep]
        df[columns_to_interpolate] = df[columns_to_interpolate].apply(pd.to_numeric, errors='coerce')
        df[columns_to_interpolate] = df[columns_to_interpolate].interpolate(method='linear', axis=0, limit_direction='both')
        df = df.reset_index()
        return df[df['time_present'] % 5 == 0].reset_index(drop=True)

    # Apply interpolation to each DataFrame
    m_present_df = m_present_df.groupby(['year', 'julian_day', 'dive_cycle'], group_keys=False).apply(interpolate_data)
    sci_time_df = sci_time_df.groupby(['year', 'julian_day', 'dive_cycle'], group_keys=False).apply(interpolate_data)
    ctd_time_df = ctd_time_df.groupby(['year', 'julian_day', 'dive_cycle'], group_keys=False).apply(interpolate_data)

    # Reset the index before concatenation
    synchronized_df = pd.concat([m_present_df.reset_index(drop=True),
                             sci_time_df.reset_index(drop=True),
                             ctd_time_df.reset_index(drop=True)], axis=1)

    # Remove duplicate columns
    synchronized_df = synchronized_df.loc[:, ~synchronized_df.columns.duplicated()]

    return synchronized_df

In [158]:
#Separate into groups to synchronise time

m_present_group = ['year', 'julian_day', 'dive_cycle','time_present', 'm_altitude', 'm_ballast_pumped', 'm_battery', 'm_battery_inst',
                                 'm_battpos', 'm_coulomb_amphr_total', 'm_depth', 'm_final_water_vx', 'm_final_water_vy',
                                 'm_lat', 'm_lon', 'm_heading', 'm_leakdetect_voltage',  'm_leakdetect_voltage_forward',
                                 'm_pitch', 'm_roll', 'm_speed']

sci_time_group = ['year', 'julian_day', 'dive_cycle', 'time_present', 'sci_time', 'sci_flbbcd_bb_units', 'sci_flbbcd_cdom_units', 'sci_flbbcd_chlor_units',
                                   'sci_oxy4_oxygen']

sci_CTD_time_group = ['year', 'julian_day', 'dive_cycle', 'time_present', 'sci_ctd_time', 'sci_water_cond', 'sci_water_pressure', 'sci_water_temp']


In [159]:
time_sync_494

,year,julian_day,dive_cycle,m_present_time,m_altitude,m_ballast_pumped,m_battery,m_battery_inst,m_battpos,m_coulomb_amphr_total,...,sci_flbbcd_cdom_units,sci_flbbcd_chlor_units,sci_m_present_time,sci_oxy4_oxygen,sci_water_cond,sci_water_pressure,sci_water_temp,time_present,sci_time,sci_ctd_time
2810,2018,37,4,1.518024e+09,10.4762,None,None,None,None,None,...,None,None,NaN,None,None,None,None,0.00000,NaN,NaN
2811,2018,37,4,1.518024e+09,None,None,None,10.9464,None,None,...,None,None,NaN,None,None,None,None,32.85528,NaN,NaN
2812,2018,37,4,1.518024e+09,None,None,None,None,None,None,...,None,None,NaN,None,None,None,None,52.22061,NaN,NaN
2813,2018,37,4,1.518024e+09,None,None,None,None,None,None,...,None,None,NaN,None,None,None,None,61.99182,NaN,NaN
2814,2018,37,4,1.518024e+09,None,None,None,None,None,None,...,None,None,NaN,None,None,None,None,86.79803,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439348,2018,123,3,1.525490e+09,None,None,None,None,None,None,...,None,None,NaN,None,None,None,None,59148.07444,NaN,NaN
1439349,2018,123,3,1.525490e+09,None,None,None,None,None,None,...,None,None,NaN,None,None,None,None,59169.20099,NaN,NaN
1439350,2018,123,3,1.525490e+09,None,None,None,None,None,None,...,None,None,NaN,None,None,None,None,59179.14188,NaN,NaN
1439351,2018,123,3,NaN,None,None,None,None,None,None,...,0,0,1.525490e+09,0,0,0,0,NaN,58976.63459,-1.525431e+09


In [160]:
synchronized_517 = process_dataset(time_sync_517, m_present_group, sci_time_group, sci_CTD_time_group)
synchronized_499 = process_dataset(time_sync_499, m_present_group, sci_time_group, sci_CTD_time_group)
synchronized_494 = process_dataset(time_sync_494, m_present_group, sci_time_group, sci_CTD_time_group)
synchronized_481 = process_dataset(time_sync_481, m_present_group, sci_time_group, sci_CTD_time_group)
synchronized_454 = process_dataset(time_sync_454, m_present_group, sci_time_group, sci_CTD_time_group)

C:\Users\Silvia\AppData\Local\Temp\ipykernel_7368\130447371.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  m_present_df = m_present_df.groupby(['year', 'julian_day', 'dive_cycle'], group_keys=False).apply(add_time_rows)
C:\Users\Silvia\AppData\Local\Temp\ipykernel_7368\130447371.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sci_time_df = sci_time_df.groupby(['year', 'julian_day', 'dive_cycle'],

In [161]:
synchronized_454

,time_present,year,julian_day,dive_cycle,m_altitude,m_ballast_pumped,m_battery,m_battery_inst,m_battpos,m_coulomb_amphr_total,...,m_pitch,m_roll,m_speed,sci_flbbcd_bb_units,sci_flbbcd_cdom_units,sci_flbbcd_chlor_units,sci_oxy4_oxygen,sci_water_cond,sci_water_pressure,sci_water_temp
0,0.0,2018,127,5,0.000000,-232.595,11.0404,11.103900,0.070001,4.02444,...,-0.431096,0.338594,0.000000,0.000842,0.1726,0.5037,398.593,3.72602,0.013,9.2074
1,5.0,2018,127,5,1.786523,-232.595,11.0404,11.103900,0.070001,4.02444,...,-0.431096,0.338594,0.006985,0.000842,0.1726,0.5037,398.593,3.72602,0.013,9.2074
2,10.0,2018,127,5,3.573047,-232.595,11.0404,11.103900,0.070001,4.02444,...,-0.431096,0.338594,0.013969,0.000842,0.1726,0.5037,398.593,3.72602,0.013,9.2074
3,15.0,2018,127,5,5.359570,-232.595,11.0404,11.103900,0.070001,4.02444,...,-0.431096,0.338594,0.020954,0.000842,0.1726,0.5037,398.593,3.72602,0.013,9.2074
4,20.0,2018,127,5,7.146093,-232.595,11.0404,11.103900,0.070001,4.02444,...,-0.431096,0.338594,0.027938,0.000842,0.1726,0.5037,398.593,3.72602,0.013,9.2074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709829,3118495.0,2018,190,0,10.285700,233.339,10.8628,10.648481,0.728170,543.34700,...,0.452040,0.083776,0.163888,0.000147,-0.1726,0.2001,327.583,3.38247,6.519,5.4024
1709830,3118500.0,2018,190,0,10.285700,233.339,10.8628,10.621089,0.728170,543.34700,...,0.452040,0.083776,0.163888,0.000147,-0.1726,0.2001,327.583,3.38247,6.519,5.4024
1709831,3118505.0,2018,190,0,10.285700,233.339,10.8628,10.607393,0.728170,543.34700,...,0.452040,0.083776,0.163888,0.000147,-0.1726,0.2001,327.583,3.38247,6.519,5.4024
1709832,3118510.0,2018,190,0,10.285700,233.339,10.8628,10.593696,0.728170,543.34700,...,0.452040,0.083776,0.163888,0.000147,-0.1726,0.2001,327.583,3.38247,6.519,5.4024


In [162]:
synchronized_517.to_parquet(r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\517\synchronized_517.parquet', index=False)
synchronized_517.to_csv(r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\517\synchronized_517.csv')

synchronized_499.to_parquet(r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\499\synchronized_499.parquet', index=False)
synchronized_499.to_csv(r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\499\synchronized_499.csv')

synchronized_494.to_parquet(r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\494\synchronized_494.parquet', index=False)
synchronized_494.to_csv(r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\494\synchronized_494.csv')

synchronized_481.to_parquet(r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\481\synchronized_481.parquet', index=False)
synchronized_481.to_csv(r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\481\synchronized_481.csv')

synchronized_454.to_parquet(r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\454\synchronized_454.parquet', index=False)
synchronized_454.to_csv(r'C:\Users\Silvia\Desktop\Year 3 Project\UG data\PREPROCESSED\454\synchronized_454.csv')

NORMALIZATION

In [163]:
# List of columns to normalize
columns_to_normalize = ['m_altitude', 'm_ballast_pumped', 'm_battery', 'm_battery_inst', 'm_battpos', 
                   'm_coulomb_amphr_total', 'm_depth', 'm_final_water_vx', 'm_final_water_vy', 'm_heading',
                   'm_lat', 'm_lon', 'm_leakdetect_voltage', 'm_leakdetect_voltage_forward', 'm_pitch', 'm_present_time', 
                   'm_roll', 'm_speed', 'sci_ctd41cp_timestamp', 'sci_flbbcd_bb_units', 'sci_flbbcd_cdom_units', 'sci_flbbcd_chlor_units',
                   'sci_m_present_time', 'sci_oxy4_oxygen', 'sci_water_cond', 'sci_water_pressure', 'sci_water_temp']

def min_max_normalize(df, columns_to_normalize=None, output_dir=None, file_name="normalized_data"):
    """
    Applies Min-Max normalization to the specified columns in the DataFrame and saves the result to Parquet and CSV.

    Args:
        df (pd.DataFrame): The input DataFrame to normalize.
        columns_to_normalize (list): List of columns to apply normalization. If None, all numeric columns will be normalized.
        output_dir (str): Directory to save the normalized files. If None, files won't be saved.
        file_name (str): Name of the output files (without extension).

    Returns:
        pd.DataFrame: The normalized DataFrame.
    """
    # If no specific columns are provided, normalize all numeric columns
    if columns_to_normalize is None:
        columns_to_normalize = df.select_dtypes(include='number').columns.tolist()

    # Apply Min-Max normalization
    for col in columns_to_normalize:
        if col in df.columns:
            df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

    # Save the normalized DataFrame to Parquet and CSV if output_dir is provided
    if output_dir:
        parquet_path = f"{output_dir}/{file_name}.parquet"
        csv_path = f"{output_dir}/{file_name}.csv"

        # Save to Parquet
        df.to_parquet(parquet_path, index=False)

        # Save to CSV
        df.to_csv(csv_path, index=False)

    return df

In [164]:
# Apply normalization and save the result
normalized_517 = min_max_normalize(synchronized_517, columns_to_normalize, Cabot_517_output_dir, "normalized_517")
normalized_499 = min_max_normalize(synchronized_499, columns_to_normalize, Dolomite_499_output_dir, "normalized_499")
normalized_494 = min_max_normalize(synchronized_494, columns_to_normalize, Stella_494_output_dir, "normalized_494")
normalized_481 = min_max_normalize(synchronized_481, columns_to_normalize, Kelvin_481_output_dir, "normalized_481")
normalized_517 = min_max_normalize(synchronized_517, columns_to_normalize, Cabot_454_output_dir, "normalized_517")

In [165]:
normalized_517

,time_present,year,julian_day,dive_cycle,m_altitude,m_ballast_pumped,m_battery,m_battery_inst,m_battpos,m_coulomb_amphr_total,...,m_pitch,m_roll,m_speed,sci_flbbcd_bb_units,sci_flbbcd_cdom_units,sci_flbbcd_chlor_units,sci_oxy4_oxygen,sci_water_cond,sci_water_pressure,sci_water_temp
0,0.0,2019.0,70.0,6.0,0.148237,0.000000,0.562588,0.880116,0.658720,0.0,...,0.193731,0.742650,0.883860,1.00000,0.695981,0.030305,0.856342,0.060190,0.001742,0.000126
1,5.0,2019.0,70.0,6.0,0.154148,0.000000,0.562588,0.880116,0.658720,0.0,...,0.193731,0.742650,0.875234,1.00000,0.695981,0.030305,0.856342,0.060190,0.001742,0.000126
2,10.0,2019.0,70.0,6.0,0.160059,0.000000,0.562588,0.880116,0.658720,0.0,...,0.193731,0.742650,0.866607,1.00000,0.695981,0.030305,0.856342,0.060190,0.001742,0.000126
3,15.0,2019.0,70.0,6.0,0.165970,0.000000,0.562588,0.880116,0.658720,0.0,...,0.193731,0.742650,0.857981,1.00000,0.695981,0.030305,0.856342,0.060190,0.001742,0.000126
4,20.0,2019.0,70.0,6.0,0.171882,0.000000,0.562588,0.880116,0.658720,0.0,...,0.193731,0.742650,0.849355,1.00000,0.695981,0.030305,0.856342,0.060190,0.001742,0.000126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311143,2492710.0,2019.0,118.0,0.0,0.184190,0.996628,0.345282,0.819662,0.942292,1.0,...,0.859150,0.775954,0.564097,0.29301,0.304492,0.030305,0.808091,0.828334,0.004152,0.822731
1311144,2492715.0,2019.0,118.0,0.0,0.184190,0.996628,0.345282,0.819662,0.942292,1.0,...,0.859150,0.775954,0.564097,0.29301,0.304492,0.030305,0.808091,0.828334,0.004152,0.822731
1311145,2492720.0,2019.0,118.0,0.0,0.184190,0.996628,0.345282,0.819662,0.942292,1.0,...,0.859150,0.775954,0.564097,0.29301,0.304492,0.030305,0.808091,0.828334,0.004152,0.822731
1311146,2492725.0,2019.0,118.0,0.0,0.184190,0.996628,0.345282,0.819662,0.942292,1.0,...,0.859150,0.775954,0.564097,0.29301,0.304492,0.030305,0.808091,0.828334,0.004152,0.822731


In [ ]:
synchronized_517 = process_dataset(time_sync_517, m_present_group, sci_time_group, sci_CTD_time_group)
synchronized_499 = process_dataset(time_sync_499, m_present_group, sci_time_group, sci_CTD_time_group)
synchronized_494 = process_dataset(time_sync_494, m_present_group, sci_time_group, sci_CTD_time_group)
synchronized_481 = process_dataset(time_sync_481, m_present_group, sci_time_group, sci_CTD_time_group)
synchronized_454 = process_dataset(time_sync_454, m_present_group, sci_time_group, sci_CTD_time_group)

C:\Users\Silvia\AppData\Local\Temp\ipykernel_7368\130447371.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  m_present_df = m_present_df.groupby(['year', 'julian_day', 'dive_cycle'], group_keys=False).apply(add_time_rows)
C:\Users\Silvia\AppData\Local\Temp\ipykernel_7368\130447371.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sci_time_df = sci_time_df.groupby(['year', 'julian_day', 'dive_cycle'],